### N-gram language models or how to write scientific papers (4 pts)

We shall train our language model on a corpora of [ArXiv](http://arxiv.org/) articles and see if we can generate a new one!

![img](https://media.npr.org/assets/img/2013/12/10/istock-18586699-monkey-computer_brick-16e5064d3378a14e0e4c2da08857efe03c04695e-s800-c85.jpg)

_data by neelshah18 from [here](https://www.kaggle.com/neelshah18/arxivdataset/)_

_Disclaimer: this has nothing to do with actual science. But it's fun, so who cares?!_

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Alternative manual download link: https://yadi.sk/d/_nGyU2IajjR9-w
!wget "https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1" -O arxivData.json.tar.gz
!tar -xvzf arxivData.json.tar.gz
data = pd.read_json("./arxivData.json")
data.sample(n=5)

--2018-10-06 14:23:17--  https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1
Распознаётся www.dropbox.com (www.dropbox.com)… 162.125.66.1
Подключение к www.dropbox.com (www.dropbox.com)|162.125.66.1|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 301 Moved Permanently
Адрес: /s/dl/99az9n1b57qkd9j/arxivData.json.tar.gz [переход]
--2018-10-06 14:23:18--  https://www.dropbox.com/s/dl/99az9n1b57qkd9j/arxivData.json.tar.gz
Повторное использование соединения с www.dropbox.com:443.
HTTP-запрос отправлен. Ожидание ответа… 302 Found
Адрес: https://uc7129fb94aadb1f22ba6753d571.dl.dropboxusercontent.com/cd/0/get/ASVDTHEmtHiqngE2WRGKt9yXz8AIJT13Bhy5b1JMxUCGQc8PNgxVNzVftXrVcD-sLbxSn00k301AbEbjPKOrI7BmxHrnQiRVQaAhxeFsDcZ8KjKFyZPu3KtyOmwqJ_NBGvzrXQoT2NyjtN1l6qMfXOL5c_7fHl89BIstERs_QvRcbXc2LFlN_mDuTLNb6aNi9AY/file?dl=1 [переход]
--2018-10-06 14:23:18--  https://uc7129fb94aadb1f22ba6753d571.dl.dropboxusercontent.com/cd/0/get/ASVDTHEmtHiqngE2WRGKt9yXz8AIJT13Bhy5b

,author,day,id,link,month,summary,tag,title,year
40562,"[{'name': 'Bin Yang'}, {'name': 'Manohar Kaul'...",2,1308.0484v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",8,We are witnessing increasing interests in the ...,"[{'term': 'cs.LG', 'scheme': 'http://arxiv.org...",Using Incomplete Information for Complete Weig...,2013
38341,"[{'name': 'Kar Wai Lim'}, {'name': 'Changyou C...",22,1609.06791v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",9,Twitter data is extremely noisy -- each tweet ...,"[{'term': 'cs.CL', 'scheme': 'http://arxiv.org...",Twitter-Network Topic Model: A Full Bayesian T...,2016
23408,"[{'name': 'Nobert Thomas Pallath'}, {'name': '...",12,1203.2404v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",3,Pedicle screw insertion technique has made rev...,"[{'term': 'cs.CV', 'scheme': 'http://arxiv.org...",Video Object Tracking and Analysis for Compute...,2012
26702,"[{'name': 'Yan Xu'}, {'name': 'Yang Li'}, {'na...",21,1611.06661v3,"[{'rel': 'related', 'href': 'http://dx.doi.org...",11,Objective: A new image instance segmentation m...,"[{'term': 'cs.CV', 'scheme': 'http://arxiv.org...",Gland Instance Segmentation Using Deep Multich...,2016
12854,"[{'name': 'Yujia Liu'}, {'name': 'Weiming Zhan...",16,1711.05934v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",11,Deep neural networks (DNNs) have achieved trem...,"[{'term': 'cs.CV', 'scheme': 'http://arxiv.org...",Enhanced Attacks on Defensively Distilled Deep...,2017


In [3]:
# assemble lines: concatenate title and description
lines = data.apply(lambda row: row['title'] + ' ; ' + row['summary'], axis=1).tolist()

sorted(lines, key=len)[:3]

['Differential Contrastive Divergence ; This paper has been retracted.',
 'What Does Artificial Life Tell Us About Death? ; Short philosophical essay',
 'P=NP ; We claim to resolve the P=?NP problem via a formal argument for P=NP.']

### Tokenization

You know the dril. The data is messy. Go clean the data. Use WordPunctTokenizer or something.


In [4]:
# Task: convert lines (in-place) into strings of space-separated tokens. import & use WordPunctTokenizer
from nltk.tokenize import WordPunctTokenizer

tokenizer = WordPunctTokenizer()

def transform_sentence(sent):
    return ' '.join(tokenizer.tokenize(sent.lower()))

lines = list(map(transform_sentence, lines))

In [5]:
assert sorted(lines, key=len)[0] == \
    'differential contrastive divergence ; this paper has been retracted .'
assert sorted(lines, key=len)[2] == \
    'p = np ; we claim to resolve the p =? np problem via a formal argument for p = np .'

### N-Gram Language Model

A language model is a probabilistic model that estimates text probability: the joint probability of all tokens $w_t$ in text $X$: $P(X) = P(w_1, \dots, w_T)$.

It can do so by following the chain rule:
$$P(w_1, \dots, w_T) = P(w_1)P(w_2 \mid w_1)\dots P(w_T \mid w_1, \dots, w_{T-1}).$$ 

The problem with such approach is that the final term $P(w_T \mid w_1, \dots, w_{T-1})$ depends on $n-1$ previous words. This probability is impractical to estimate for long texts, e.g. $T = 1000$.

One popular approximation is to assume that next word only depends on a finite amount of previous words:

$$P(w_{t + 1} \mid w_1, \dots, w_{t - 1}) = P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1})$$

Such model is called __n-gram language model__ where n is a parameter. For example, in 3-gram language model, each word only depends on 2 previous words. 

$$
    P(w_1, \dots, w_n) = \prod_t P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1}).
$$

You can also sometimes see such approximation under the name of _n-th order markov assumption_.

The first stage to building such a model is counting all word occurences given N-1 previous words

In [6]:
from tqdm import tqdm
from collections import defaultdict, Counter

# special tokens: 
# - unk represents absent tokens, 
# - eos is a special token after the end of sequence

UNK, EOS = "_UNK_", "_EOS_"

def count_ngrams(lines, n):
    """
    Count how many times each word occured after (n - 1) previous words
    :param lines: an iterable of strings with space-separated tokens
    :returns: a dictionary { tuple(prefix_tokens): {next_token_1: count_1, next_token_2: count_2}}

    When building counts, please consider the following two edge cases
    - if prefix is shorter than (n - 1) tokens, it should be padded with UNK. For n=3,
      empty prefix: "" -> (UNK, UNK)
      short prefix: "the" -> (UNK, the)
      long prefix: "the new approach" -> (new, approach)
    - you should add a special token, EOS, at the end of each sequence
      "... with deep neural networks ." -> (..., with, deep, neural, networks, ., EOS)
      count the probability of this token just like all others.
    """
    counts = defaultdict(Counter)
    # counts[(word1, word2)][word3] = how many times word3 occured after (word1, word2)
    for line in tqdm(lines):
        line = (UNK + ' ') * (n-1) + line + ' ' + EOS
        tokens = line.split()
        
        for idx in range(n - 1, len(tokens)):
            prefix = tokens[idx - n + 1: idx]
            counts[tuple(prefix)][tokens[idx]] += 1
    
    return counts

In [7]:
# let's test it
dummy_lines = sorted(lines, key=len)[:100]
dummy_counts = count_ngrams(dummy_lines, n=3)
assert set(map(len, dummy_counts.keys())) == {2}, "please only count {n-1}-grams"
assert len(dummy_counts[('_UNK_', '_UNK_')]) == 78
assert dummy_counts['_UNK_', 'a']['note'] == 3
assert dummy_counts['p', '=']['np'] == 2
assert dummy_counts['author', '.']['_EOS_'] == 1

100%|██████████| 100/100 [00:00<00:00, 13959.14it/s]


Once we can count N-grams, we can build a probabilistic language model.
The simplest way to compute probabilities is in proporiton to counts:

$$ P(w_t | prefix) = { Count(prefix, w_t) \over \sum_{\hat w} Count(prefix, \hat w) } $$

In [8]:
class NGramLanguageModel:
    def __init__(self, lines, n):
        """ 
        Train a simple count-based language model: 
        compute probabilities P(w_t | prefix) given ngram counts
        
        :param n: computes probability of next token given (n - 1) previous words
        :param lines: an iterable of strings with space-separated tokens
        """
        assert n >= 1
        self.n = n
    
        counts = count_ngrams(lines, self.n)
        
        # compute token proabilities given counts
        self.probs = defaultdict(Counter)
        # probs[(word1, word2)][word3] = P(word3 | word1, word2)
        
        # populate self.probs with actual probabilities
        for prefix, token_counts in counts.items():
            prefix_count = sum(token_counts.values())
            for token, count in token_counts.items():
                # should fix dividing by 0
                self.probs[prefix][token] = count / max(prefix_count, 1)
        
            
    def get_possible_next_tokens(self, prefix):
        """
        :param prefix: string with space-separated prefix tokens
        :returns: a dictionary {token : it's probability} for all tokens with positive probabilities
        """
        prefix = prefix.split()
        prefix = prefix[max(0, len(prefix) - self.n + 1):]
        prefix = [ UNK ] * (self.n - 1 - len(prefix)) + prefix
        return self.probs[tuple(prefix)]
    
    def get_next_token_prob(self, prefix, next_token):
        """
        :param prefix: string with space-separated prefix tokens
        :param next_token: the next token to predict probability for
        :returns: P(next_token|prefix) a single number, 0 <= P <= 1
        """
        return self.get_possible_next_tokens(prefix).get(next_token, 0)

Let's test it!

In [9]:
dummy_lm = NGramLanguageModel(dummy_lines, n=3)

p_initial = dummy_lm.get_possible_next_tokens('') # '' -> ['_UNK_', '_UNK_']
assert np.allclose(p_initial['learning'], 0.02)
assert np.allclose(p_initial['a'], 0.13)
assert np.allclose(p_initial.get('meow', 0), 0)
assert np.allclose(sum(p_initial.values()), 1)

p_a = dummy_lm.get_possible_next_tokens('a') # '' -> ['_UNK_', 'a']
assert np.allclose(p_a['machine'], 0.15384615)
assert np.allclose(p_a['note'], 0.23076923)
assert np.allclose(p_a.get('the', 0), 0)
assert np.allclose(sum(p_a.values()), 1)

assert np.allclose(dummy_lm.get_possible_next_tokens('a note')['on'], 1)
assert dummy_lm.get_possible_next_tokens('a machine') == \
    dummy_lm.get_possible_next_tokens("there have always been ghosts in a machine"), \
    "your 3-gram model should only depend on 2 previous words"

100%|██████████| 100/100 [00:00<00:00, 13088.79it/s]


Now that you've got a working n-gram language model, let's see what sequences it can generate. But first, let's train it on the whole dataset.

In [10]:
lm = NGramLanguageModel(lines, n=3)

100%|██████████| 41000/41000 [00:17<00:00, 2369.39it/s]


The process of generating sequences is... well, it's sequential. You maintain a list of tokens and iteratively add next token by sampling with probabilities.

$ X = [] $

__forever:__
* $w_{next} \sim P(w_{next} | X)$
* $X = concat(X, w_{next})$


Instead of sampling with probabilities, one can also try always taking most likely token, sampling among top-K most likely tokens or sampling with temperature. In the latter case (temperature), one samples from

$$w_{next} \sim {P(w_{next} | X) ^ {1 / \tau} \over \sum_{\hat w} P(\hat w | X) ^ {1 / \tau}}$$

Where $\tau > 0$ is model temperature. If $\tau << 1$, more likely tokens will be sampled with even higher probability while less likely tokens will vanish.

In [11]:
def get_next_token(lm, prefix, temperature=1.0):
    """
    return next token after prefix;
    :param temperature: samples proportionally to lm probabilities ^ temperature
        if temperature == 0, always takes most likely token. Break ties arbitrarily.
    """
    
    # TODO: should also fix problem with empty possible next tokens
    tokens, probs = zip(*lm.get_possible_next_tokens(prefix).items())
    
    if temperature == 0:
        return tokens[np.argmax(probs)]
    probs = np.array(probs) ** (1. / temperature)
    probs /= np.sum(probs)
    
    return np.random.choice(tokens, p=probs)

In [12]:
from collections import Counter
test_freqs = Counter([get_next_token(lm, 'there have') for _ in range(10000)])
assert 250 < test_freqs['not'] < 450
assert 8500 < test_freqs['been'] < 9500
assert 1 < test_freqs['lately'] < 200

test_freqs = Counter([get_next_token(lm, 'deep', temperature=1.0) for _ in range(10000)])
assert 1500 < test_freqs['learning'] < 3000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.5) for _ in range(10000)])
assert 8000 < test_freqs['learning'] < 9000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.0) for _ in range(10000)])
assert test_freqs['learning'] == 10000

print("Looks nice!")

Looks nice!


Let's have fun with this model

In [21]:
prefix = 'movie' # <- your ideas :)

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break

print(prefix)

movie recommendation problem addresses detecting the discontinuity can be formulated . here we investigated a c - arm bandits and is twice the average setting , which can be analysed and classified microstructures separately from other counts , as well as simulated annealing ( asa ) to select a representative set of 99 . 10 and cifar - 100 and above the gilbert and mdm algorithms . in principle , offers significant speed - up rate . however , the single - pass learning strategy can provide clear insight in this paper , we introduce an efficient self - repairable web


In [22]:
prefix = 'sentence' # <- more of your ideas

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix, temperature=0.5)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

sentence - level labels ; in this paper , we propose a novel approach to learning the parameters of the proposed method is able to automatically learn similarity information in the literature , and we show that the proposed method is based on a variety of applications in computer vision . in this paper , we present a new algorithm for computing the kernel distance , and we present a novel method to the problem of estimating the latent variables . we also present a novel hybrid densely connected unet for liver and the local statistics of any network size by


### Evaluating language models: perplexity

Perplexity is a measure of how well does your model approximate true probability distribution behind data. __Smaller perplexity = better model__.

To compute perplexity on one sentence, use:
$$
    {\mathbb{P}}(w_1 \dots w_N) = P(w_1, \dots, w_N)^{-\frac1N} = \left( \prod_t P(w_t \mid w_{t - n}, \dots, w_{t - 1})\right)^{-\frac1N},
$$


On the corpora level, perplexity is a product of probabilities of all tokens in all sentences to the power of 1, divided by __total length of all sentences__ in corpora.

This number can quickly get too small for float32/float64 precision, so we recommend you to first compute log-perplexity (from log-probabilities) and then take the exponent.

In [23]:
def perplexity(lm, lines, min_logprob=np.log(10 ** -50.)):
    """
    :param lines: a list of strings with space-separated tokens
    :param min_logprob: if log(P(w | ...)) is smaller than min_logprop, set it equal to min_logrob
    :returns: corpora-level perplexity - a single scalar number from the formula above
    
    Note: do not forget to compute P(w_first | empty) and P(eos | full_sequence)
    
    PLEASE USE lm.get_next_token_prob and NOT lm.get_possible_next_tokens
    """
    n = lm.n
    
    prob = 0
    tokens_count = 0
    for line in lines:
        tokens = line.split()
        tokens_count += len(tokens) + 1
        tokens = [UNK] * (n-1) + tokens + [EOS]
        for idx in range(n - 1, len(tokens)):
            prefix, token = tokens[idx - n + 1: idx], tokens[idx]
            next_token_prob = lm.get_next_token_prob(' '.join(prefix), token)
            # such construction for avoiding zero div
            prob += min_logprob if next_token_prob == 0 else max(np.log(next_token_prob), min_logprob)
    return np.exp(-1. / tokens_count * prob)

In [24]:
lm1 = NGramLanguageModel(dummy_lines, n=1)
lm3 = NGramLanguageModel(dummy_lines, n=3)
lm10 = NGramLanguageModel(dummy_lines, n=10)

ppx1 = perplexity(lm1, dummy_lines)
ppx3 = perplexity(lm3, dummy_lines)
ppx10 = perplexity(lm10, dummy_lines)
ppx_missing = perplexity(lm3, ['the jabberwock , with eyes of flame , '])  # thanks, L. Carrol

print("Perplexities: ppx1=%.3f ppx3=%.3f ppx10=%.3f" % (ppx1, ppx3, ppx10))

assert all(0 < ppx < 500 for ppx in (ppx1, ppx3, ppx10)), "perplexity should be nonnegative and reasonably small"
assert ppx1 > ppx3 > ppx10, "higher N models should overfit and "
assert np.isfinite(ppx_missing) and ppx_missing > 10 ** 6, "missing words should have large but finite perplexity. " \
    " Make sure you use min_logprob right"
assert np.allclose([ppx1, ppx3, ppx10], (318.2132342216302, 1.5199996213739575, 1.1838145037901249))

100%|██████████| 100/100 [00:00<00:00, 10489.96it/s]

Perplexities: ppx1=318.213 ppx3=1.520 ppx10=1.184


Now let's measure the actual perplexity: we'll split the data into train and test and score model on test data only.

In [25]:
from sklearn.model_selection import train_test_split
train_lines, test_lines = train_test_split(lines, test_size=0.25, random_state=42)

for n in (1, 2, 3):
    lm = NGramLanguageModel(n=n, lines=train_lines)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

  1%|▏         | 408/30750 [00:00<00:07, 4072.80it/s]

N = 1, Perplexity = 1832.23136


  1%|          | 250/30750 [00:00<00:12, 2499.60it/s]

N = 2, Perplexity = 85653987.28774


100%|██████████| 30750/30750 [00:13<00:00, 2196.84it/s]

N = 3, Perplexity = 61999196259042902147072.00000


In [ ]:
# whoops, it just blew up :)

### LM Smoothing

The problem with our simple language model is that whenever it encounters an n-gram it has never seen before, it assigns it with the probabilitiy of 0. Every time this happens, perplexity explodes.

To battle this issue, there's a technique called __smoothing__. The core idea is to modify counts in a way that prevents probabilities from getting too low. The simplest algorithm here is Additive smoothing (aka [Lapace smoothing](https://en.wikipedia.org/wiki/Additive_smoothing)):

$$ P(w_t | prefix) = { Count(prefix, w_t) + \delta \over \sum_{\hat w} (Count(prefix, \hat w) + \delta) } $$

If counts for a given prefix are low, additive smoothing will adjust probabilities to a more uniform distrivution. Not that the summation in the denominator goes over _all words in the vocabulary_.

Here's an example code we've implemented for you:

In [26]:
class LaplaceLanguageModel(NGramLanguageModel):
    """ this code is an example, no need to change anything """
    def __init__(self, lines, n, delta=1.0):
        self.n = n
        counts = count_ngrams(lines, self.n)
        self.vocab = set(token for token_counts in counts.values() for token in token_counts)
        self.probs = defaultdict(Counter)

        for prefix in counts:
            token_counts = counts[prefix]
            total_count = sum(token_counts.values()) + delta * len(self.vocab)
            self.probs[prefix] = {token: (token_counts[token] + delta) / total_count
                                          for token in token_counts}
    def get_possible_next_tokens(self, prefix):
        token_probs = super().get_possible_next_tokens(prefix)
        missing_prob_total = 1.0 - sum(token_probs.values())
        missing_prob = missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        return {token: token_probs.get(token, missing_prob) for token in self.vocab}
    
    def get_next_token_prob(self, prefix, next_token):
        token_probs = super().get_possible_next_tokens(prefix)
        if next_token in token_probs:
            return token_probs[next_token]
        else:
            missing_prob_total = 1.0 - sum(token_probs.values())
            missing_prob_total = max(0, missing_prob_total) # prevent rounding errors
            return missing_prob_total / max(1, len(self.vocab) - len(token_probs))

In [27]:
#test that it's a valid probability model
for n in (1, 2, 3):
    dummy_lm = LaplaceLanguageModel(dummy_lines, n=n)
    assert np.allclose(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]), 1), "I told you not to break anything! :)"

100%|██████████| 100/100 [00:00<00:00, 12210.14it/s]


In [28]:
for n in (1, 2, 3):
    lm = LaplaceLanguageModel(train_lines, n=n, delta=0.1)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

  1%|▏         | 420/30750 [00:00<00:07, 4193.42it/s]

N = 1, Perplexity = 977.67559


  1%|          | 253/30750 [00:00<00:12, 2524.90it/s]

N = 2, Perplexity = 470.48021


100%|██████████| 30750/30750 [00:14<00:00, 2167.19it/s]

N = 3, Perplexity = 3679.44765


In [56]:
# optional: try to sample tokens from such a model
prefix = 'simple question'

for i in range(50):
    # with temperature > 0, it is very easy to get sentence without any sense
    prefix += ' ' + get_next_token(lm, prefix, 0)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break

print(prefix)

simple question answering ( vqa ) is a challenging task . _EOS_


### Kneser-Ney smoothing

Additive smoothing is simple, reasonably good but definitely not a State of The Art algorithm.


Your final task in this notebook is to implement [Kneser-Ney](https://en.wikipedia.org/wiki/Kneser%E2%80%93Ney_smoothing) smoothing.

It can be computed recurrently, for n>1:

$$P_{kn}(w_t | prefix_{n-1}) = { \max(0, Count(prefix_{n-1}, w_t) - \delta) \over \sum_{\hat w} \max(0, Count(prefix_{n-1}, \hat w))} + \lambda_{prefix_{n-1}} \cdot P_{kn}(w_t | prefix_{n-2})$$

where
- $prefix_{n-1}$ is a tuple of {n-1} previous tokens
- $lambda_{prefix_{n-1}}$ is a normalization constant chosen so that probabilities add up to 1
- Unigram $P_{kn}(w_t | prefix_{n-2})$ corresponds to Kneser Ney smoothing for {N-1}-gram language model.
- Unigram $P_{kn}(w_t)$ is a special case: how likely it is to see x_t in an unfamiliar context

See lecture slides or wiki for more detailed formulae.

__Your task__ is to
- implement KneserNeyLanguageModel
- test it on 1-3 gram language models
- find optimal (within one order of magnitude) smoothing delta for 3-gram language model with Kneser-Ney smoothing

In [155]:
class KneserNeyLanguageModel(NGramLanguageModel):
    """ this code is an example, no need to change anything """
    def __init__(self, lines, n, delta=1.0):
        self.n = n
        self._delta = delta
        
        self.counts = {} # key is (prefix size + 1)
        for i in range(1, max(3, n+1)):
            self.counts[i] = count_ngrams(lines, i)
        self.vocab = set(token for token_counts in self.counts[n].values() for token in token_counts)
        
        self._prefixes = Counter() # number of pairs with w_i after: (*w_i)
        for prefix, token_counts in self.counts[2].items():
            self._prefixes.update(token_counts.keys())
        self._bigrams_counter = sum(self._prefixes.values()) #: (**)
        
        self._postfixes = Counter() # number of words after concrete prefix: (prefix*)
        for i in range(2, n+1):
            for prefix, token_counts in self.counts[i].items():
                self._postfixes[prefix] = len(token_counts)
    
    def _get_prob(self, prefix, token):
        if len(prefix) == 0:
            return 1. * self._prefixes[token] / self._bigrams_counter
        d = self._delta
        denominator = sum(self.counts[len(prefix)+1][prefix].values())
        numerator = self.counts[len(prefix)+1][prefix][token] if denominator else 0
        N1_plus = self._postfixes[prefix]
        if denominator == 0:
            return self._get_prob(prefix[1:], token)
        return max(numerator - d, 0) / denominator + d / denominator * N1_plus * self._get_prob(prefix[1:], token)

    def get_possible_next_tokens(self, prefix):
        return {token: self.get_next_token_prob(prefix, token) for token in self.vocab}

    def get_next_token_prob(self, prefix, next_token):
        prefix = prefix.split()
        prefix = prefix[max(0, len(prefix) - self.n + 1):]
        prefix = [ UNK ] * (self.n - 1 - len(prefix)) + prefix

        return self._get_prob(tuple(prefix), next_token)

In [151]:
#test that it's a valid probability model
for n in (1, 2, 3):
    lm = KneserNeyLanguageModel(dummy_lines, n=n)
    assert np.allclose(sum([lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]), 1), "I told you not to break anything! :)"

100%|██████████| 100/100 [00:00<00:00, 11782.41it/s]


In [153]:
for n in (1, 2, 3):
    lm = KneserNeyLanguageModel(train_lines, n=n)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

  2%|▏         | 660/30750 [00:00<00:04, 6597.82it/s]

N = 1, Perplexity = 2885.69413


  2%|▏         | 645/30750 [00:00<00:04, 6447.29it/s]

N = 2, Perplexity = 404.35703


100%|██████████| 30750/30750 [00:14<00:00, 2114.20it/s]

N = 3, Perplexity = 311.53726


In [157]:
lm = KneserNeyLanguageModel(train_lines, n=3)

100%|██████████| 30750/30750 [00:13<00:00, 2208.04it/s]


In [164]:
# Check this model also:
prefix = 'context recognition'
for i in range(30):
    prefix += ' ' + get_next_token(lm, prefix)
    if prefix.endswith(EOS):
        break

print(prefix)

context recognition systems for fast noise reduction approach has the dr . the second stage improves on the signal to be highly loop criteriea offers candidates for controlling the neural response to
